In [1]:
import os
import torch
import torch.utils.data
import pandas as pd
import numpy as np
from torch import nn, optim
from torch import Tensor
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset



In [12]:
CUDA = True
SEED = 1
BATCH_SIZE = 1
LOG_INTERVAL = 10
EPOCHS = 50
VALIDATION_SPLIT = .2
ZDIMS = 60
ENCODER_DIMS = 6000
DECODER_DIMS = 6000

In [3]:
torch.manual_seed(SEED)
device = torch.device("cuda" if CUDA else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}

In [ ]:
#train_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
#    batch_size=BATCH_SIZE, shuffle=True, **kwargs

#)

#test_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
#    batch_size=BATCH_SIZE, shuffle=True, **kwargs)               




In [ ]:
import pandas as pd

kmer_path = "data\kmers-gzip\\output.txt.gz"

kmer_all = pd.read_csv(kmer_path ,delim_whitespace=True, compression="gzip", names = ["kmer", "count"], index_col = 0)

#kmers = kmer_all.set_index("kmer", drop=False)




In [ ]:
kmer_culled = kmer_all.loc[kmer_all['count'] >= 700].copy()
print(kmer_all.describe())

print(kmer_culled.describe())




In [ ]:

kmer_path = "data\kmers-gzip\\s314.txt.gz"
kmer_path2 = "data\kmers-gzip\\upec-261.txt.gz"


kmer_contig1 = pd.read_csv(kmer_path ,delim_whitespace=True, compression="gzip", names = ["kmer", "count"], index_col = 0)
kmer_contig2 = pd.read_csv(kmer_path2 ,delim_whitespace=True, compression="gzip", names = ["kmer", "count"], index_col = 0)


kmer_contig1.sort_values(by="kmer", inplace=True)


contig1_parsed = kmer_contig1[kmer_contig1.index.isin(kmer_culled.index)]
contig2_parsed = kmer_contig2[kmer_contig2.index.isin(kmer_culled.index)]


kmer_temp = kmer_culled.copy()
kmer_temp['count'] = '0'
kmer_temp.sort_values(by='kmer', inplace=True)

print(contig1_parsed)
print(kmer_temp.head(70))
print(kmer_temp.loc["AAAAACTCTGCTTACCAGGCGCATTTCGCCC"])

In [ ]:
len(kmer_temp)

In [ ]:
contig1_merged = pd.merge(contig1_parsed, kmer_temp, how='right',on='kmer')
contig2_merged = contig2_parsed.merge(kmer_temp, how='right',on='kmer')

contig_noNull = contig1_merged[[ 'count_x']].fillna(value='0').copy()
contig_noNull2 = contig2_merged[[ 'count_x']].fillna(value='0').copy()
pd.isnull(contig_noNull)


contig_noNull.sort_values(by=['kmer'], inplace=True)
contig_noNull2.sort_values(by=['kmer'], inplace=True)

#print(kmer_temp.describe())

print(contig_noNull.head(10))
print(contig_noNull2.head(10))
#contig_clean = contig1_merged[['kmer', 'count_x']].copy()
#contig_clean.head()
#pd.isnull(contig_clean)


In [4]:


CULL_SIZE = 700

class KmerDataset(Dataset):
    
    def __init__(self, dirname):
        files = os.listdir(dirname)
        max_kmer = 0
        X, y = [],[]
        kmer_all = pd.read_csv('data\kmers-gzip\\output.txt.gz' ,delim_whitespace=True, compression="gzip", names = ["kmer", "count"])
        kmer_culled = kmer_all.loc[kmer_all['count'] >= CULL_SIZE]
        kmer_temp = kmer_culled.copy()
        kmer_temp['count'] = '0'
        kmer_temp.sort_values(by=['kmer'], inplace=True)
        self.template = kmer_temp
        self.feature_dim = len(kmer_temp)
        for line in files:
            if(line != 'output.txt.gz'):

                kmer_path = dirname + '\\' + line
                kmer_df = pd.read_csv(kmer_path ,delim_whitespace=True, compression="gzip", names = ["kmer", "count"])
                contig_parsed = kmer_df[kmer_df.kmer.isin(self.template.kmer)]
                contig_merged = contig_parsed.merge(self.template, how='right',on='kmer')

                contig_final = contig_merged[['kmer', 'count_x']].fillna(value=0).copy()
                contig_final.sort_values(by=['kmer'], inplace=True)
                contig_vec = torch.tensor(contig_final['count_x'].values, dtype=torch.float)
                X.append(contig_vec)
                contig_max = torch.max(contig_vec)
                if(contig_max > max_kmer): max_kmer = contig_max
                
        self.X = X
        self.max_kmer = max_kmer
    
    def preprocess(self, contig):
        
        norm_vec = torch.div(contig, self.max_kmer)

        #print(contig_final.head(10))
        
        #print(np.isnan(contig_vec))
        
        return norm_vec
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, index):
        
        return self.preprocess(self.X[index])

In [5]:
#initialize dataset
dataset = KmerDataset('data\kmers-gzip')



In [ ]:
dataset[100]

In [6]:
#Separate into train/val

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(VALIDATION_SPLIT * dataset_size))
np.random.seed(SEED)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

#initialize data loaders

train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, 
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,
                                                sampler=valid_sampler)
FEATURE_SIZE = dataset.feature_dim
print(FEATURE_SIZE)

8763


In [ ]:
len(test_loader)

In [7]:

#Variable is deprecated update at some point

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(FEATURE_SIZE, ENCODER_DIMS)
        
        self.relu = nn.ReLU()
        self.fc21 = nn.Linear(ENCODER_DIMS, ZDIMS)
        self.fc22 = nn.Linear(ENCODER_DIMS, ZDIMS)
        
        
        self.fc3 = nn.Linear(ZDIMS, DECODER_DIMS)
        
        self.fc4 = nn.Linear(DECODER_DIMS, FEATURE_SIZE)
        self.sigmoid = nn.Sigmoid()
        
    
    #if in training creates random vector based on mean and stddev
    #if in otherwise returns constant mean for backprop
    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        
        if self.training:
            
            std = logvar.mul(.5).exp_()
            
            eps = std.data.new(std.size()).normal_()
            
            return eps.mul(std).add_(mu)
        else:
            return mu
    
    def encode(self, x: Tensor) -> (Tensor, Tensor):
        
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def decode(self, z: Tensor) -> Tensor:
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x: Tensor) -> (Tensor, Tensor, Tensor):
        #feed forward for the network
        
        #encodes data into mean(mu) and logvarience(logvar, ln(sigma^2))
        mu, logvar = self.encode(x.view(-1, FEATURE_SIZE))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

        

In [8]:

model = VAE().to(device)



def loss_function(recon_x, x, mu, logvar):
    
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, FEATURE_SIZE))
    
    
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    KLD /= BATCH_SIZE * FEATURE_SIZE
    
    return BCE + KLD



In [13]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train(epoch):
    
    model.train()
    train_loss = 0
    
    for i in enumerate(train_loader):
        data = i[1]
        batch_idx = i[0]
        data = data.to(device)
        
        optimizer.zero_grad()
        
        recon_batch, mu, logvar = model(data)
        
        loss = loss_function(recon_batch, data, mu, logvar)
        
        loss.backward()
        
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len (train_loader.dataset),
            100. * batch_idx / len(train_loader),
            loss.data.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)
    ))

In [14]:
def test(epoch):
    
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i in enumerate(test_loader):
            data = i[1]
            if CUDA:
                data = data.cuda()
        
        
                recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).data.item()
            torch.cuda.empty_cache
        
        test_loss /= len(test_loader.dataset)
        print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
torch.cuda.empty_cache()

In [15]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test(epoch)
    
    
    sample = torch.randn(64, ZDIMS).to(device)
    
   
    
    
    sample = model.decode(sample).cpu()
    
    
    #save_image(sample.data.view(64, 1, 28, 28),
              #'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/347 (0%)]	Loss: 0.029000
Train Epoch: 1 [10/347 (4%)]	Loss: 0.064315
Train Epoch: 1 [20/347 (7%)]	Loss: 0.025125
Train Epoch: 1 [30/347 (11%)]	Loss: 0.085514
Train Epoch: 1 [40/347 (14%)]	Loss: 0.087538
Train Epoch: 1 [50/347 (18%)]	Loss: 0.069661
Train Epoch: 1 [60/347 (22%)]	Loss: 0.033249
Train Epoch: 1 [70/347 (25%)]	Loss: 0.052722
Train Epoch: 1 [80/347 (29%)]	Loss: 0.067163
Train Epoch: 1 [90/347 (32%)]	Loss: 0.119969
Train Epoch: 1 [100/347 (36%)]	Loss: 0.060958
Train Epoch: 1 [110/347 (40%)]	Loss: 0.023962
Train Epoch: 1 [120/347 (43%)]	Loss: 0.022500
Train Epoch: 1 [130/347 (47%)]	Loss: 0.032701
Train Epoch: 1 [140/347 (50%)]	Loss: 0.033330
Train Epoch: 1 [150/347 (54%)]	Loss: 0.056419
Train Epoch: 1 [160/347 (58%)]	Loss: 0.037024
Train Epoch: 1 [170/347 (61%)]	Loss: 0.038130
Train Epoch: 1 [180/347 (65%)]	Loss: 0.123158
Train Epoch: 1 [190/347 (68%)]	Loss: 0.022922
Train Epoch: 1 [200/347 (72%)]	Loss: 0.028078
Train Epoch: 1 [210/347 (76%)]	Loss: 0.043082
Tr

Train Epoch: 7 [40/347 (14%)]	Loss: 0.022167
Train Epoch: 7 [50/347 (18%)]	Loss: 0.042624
Train Epoch: 7 [60/347 (22%)]	Loss: 0.101970
Train Epoch: 7 [70/347 (25%)]	Loss: 0.032753
Train Epoch: 7 [80/347 (29%)]	Loss: 0.054882
Train Epoch: 7 [90/347 (32%)]	Loss: 0.036702
Train Epoch: 7 [100/347 (36%)]	Loss: 0.080164
Train Epoch: 7 [110/347 (40%)]	Loss: 0.050898
Train Epoch: 7 [120/347 (43%)]	Loss: 0.033298
Train Epoch: 7 [130/347 (47%)]	Loss: 0.031640
Train Epoch: 7 [140/347 (50%)]	Loss: 0.055590
Train Epoch: 7 [150/347 (54%)]	Loss: 0.024361
Train Epoch: 7 [160/347 (58%)]	Loss: 0.035623
Train Epoch: 7 [170/347 (61%)]	Loss: 0.033730
Train Epoch: 7 [180/347 (65%)]	Loss: 0.024511
Train Epoch: 7 [190/347 (68%)]	Loss: 0.050774
Train Epoch: 7 [200/347 (72%)]	Loss: 0.119904
Train Epoch: 7 [210/347 (76%)]	Loss: 0.021593
Train Epoch: 7 [220/347 (79%)]	Loss: 0.028144
Train Epoch: 7 [230/347 (83%)]	Loss: 0.028783
Train Epoch: 7 [240/347 (86%)]	Loss: 0.048872
Train Epoch: 7 [250/347 (90%)]	Loss: 0.0

Train Epoch: 13 [60/347 (22%)]	Loss: 0.027838
Train Epoch: 13 [70/347 (25%)]	Loss: 0.034875
Train Epoch: 13 [80/347 (29%)]	Loss: 0.026957
Train Epoch: 13 [90/347 (32%)]	Loss: 0.064213
Train Epoch: 13 [100/347 (36%)]	Loss: 0.047201
Train Epoch: 13 [110/347 (40%)]	Loss: 0.041801
Train Epoch: 13 [120/347 (43%)]	Loss: 0.076802
Train Epoch: 13 [130/347 (47%)]	Loss: 0.040739
Train Epoch: 13 [140/347 (50%)]	Loss: 0.085755
Train Epoch: 13 [150/347 (54%)]	Loss: 0.038375
Train Epoch: 13 [160/347 (58%)]	Loss: 0.023113
Train Epoch: 13 [170/347 (61%)]	Loss: 0.045282
Train Epoch: 13 [180/347 (65%)]	Loss: 0.039898
Train Epoch: 13 [190/347 (68%)]	Loss: 0.060926
Train Epoch: 13 [200/347 (72%)]	Loss: 0.107674
Train Epoch: 13 [210/347 (76%)]	Loss: 0.047694
Train Epoch: 13 [220/347 (79%)]	Loss: 0.075841
Train Epoch: 13 [230/347 (83%)]	Loss: 0.066412
Train Epoch: 13 [240/347 (86%)]	Loss: 0.075773
Train Epoch: 13 [250/347 (90%)]	Loss: 0.033429
Train Epoch: 13 [260/347 (94%)]	Loss: 0.066651
Train Epoch: 13 [

Train Epoch: 19 [60/347 (22%)]	Loss: 0.100765
Train Epoch: 19 [70/347 (25%)]	Loss: 0.030518
Train Epoch: 19 [80/347 (29%)]	Loss: 0.072236
Train Epoch: 19 [90/347 (32%)]	Loss: 0.063315
Train Epoch: 19 [100/347 (36%)]	Loss: 0.037994
Train Epoch: 19 [110/347 (40%)]	Loss: 0.027756
Train Epoch: 19 [120/347 (43%)]	Loss: 0.058548
Train Epoch: 19 [130/347 (47%)]	Loss: 0.066994
Train Epoch: 19 [140/347 (50%)]	Loss: 0.053182
Train Epoch: 19 [150/347 (54%)]	Loss: 0.045257
Train Epoch: 19 [160/347 (58%)]	Loss: 0.044713
Train Epoch: 19 [170/347 (61%)]	Loss: 0.053094
Train Epoch: 19 [180/347 (65%)]	Loss: 0.051200
Train Epoch: 19 [190/347 (68%)]	Loss: 0.028865
Train Epoch: 19 [200/347 (72%)]	Loss: 0.028017
Train Epoch: 19 [210/347 (76%)]	Loss: 0.030419
Train Epoch: 19 [220/347 (79%)]	Loss: 0.032222
Train Epoch: 19 [230/347 (83%)]	Loss: 0.077111
Train Epoch: 19 [240/347 (86%)]	Loss: 0.077085
Train Epoch: 19 [250/347 (90%)]	Loss: 0.028850
Train Epoch: 19 [260/347 (94%)]	Loss: 0.025925
Train Epoch: 19 [

KeyboardInterrupt: 

10 Epochs each w/ batch size of 1
w/ 10 zdims and 1000 hidden layer: Test set loss: 0.0097
w/ 3000 zdims and 5000 hidden layers: Test set loss: 0.0136
w/ 1000 zdims and 3000 hidden layers: Test set loss: like .02
w/ 300 zdims and 1000 hidden layers: Test set loss: 0.0333